DESCRIPTION DU PROJET:

Nous allons dans ce travail pratique effectuer la classification de visage en utilisant la combinaison LBP BASIQUE + SVM

LBP nous servira pour l'extraction des caractéristiques tandisque SVM sert à la classification.

Le jeu de données utilisé provient d'un laboratoire Anglais, il s'agit de 40 personnes filmées 10 fois chacunes puis 

regroupés au sein d'un dossier , l'ensemble contenus dans un dossier parent. Nous avons donc exactement 40*10=400 images au format PGM

binaire et toutes de dimensions égales(92*112)

Les étapes de ce projet sont :

1- Collecte de données(déja décrit précedemment)   

2- prétraitement : ici nos images sont déja en niveau de gris et de tailles égales , donc plus besoin de convertir ou de redimensionner

3- Extraction des caractéristiques : nous nous servirons de l'approche LBP BASIQUE;
pour une image quelconque , recupérer sa matrice de pixels, déterminer la matrice LBP correspondante puis l'histogramme de l'image LBP

4- Séparation des données en train et test: les données ici seront nos histogrammes

5- Entrainement du modèle: nous allons entrainer notre SVM sur les données de train obtenues après séparation

6- Amélioration du modèle: nous nous servirons de l'approche grid search pour peaufiner notre modèle

7- Classification de nouvelles images: nous définirons une approche de classification de nouvelles de données en utilisant le modèles amélioré 

1- IMPORTATION DES BIBLIOTHEQUES

In [1]:
import os
from PIL import Image
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import cross_val_score
import pickle

2- DEFINITION DE LA FONCTION QUI PREND EN ENTREE UNE IMAGE ET RETOURNE SON VECTEUR DE PIXELS

In [2]:
def extract(imag):
    with open(imag , 'rb') as f:   #ouvrir le fichier en mode lecture binaire
        magic_num = f.readline().strip()     # strip() pour supprimer les espaces 
        width , height = map(int , f.readline().split())
        max_val = int(f.readline())
        pixls =[]
        for _ in range(width):
            row = []
            for _ in range(height):
                byte = f.read(1)
                pix_val = int.from_bytes(byte , byteorder='big')
                row.append(pix_val)
            pixls.append(row)
    return np.array(pixls)

In [3]:
print(extract('archive (1)/s1/1.pgm').shape)

(92, 112)


2- EXTRACTIOONS DES MATRICES DE PIXELS DE TOUTES NOS DONNÉES

In [4]:


def Lecture_fichiers(chemin_acces):
    images = []
    target = []
    for folder_name in os.listdir(chemin_acces):
        folder_path = os.path.join(chemin_acces , folder_name)
        if os.path.isdir(folder_path):
            for image_name in os.listdir(folder_path):
                image_path = os.path.join(folder_path , image_name)
                if image_path.endswith('.pgm'):
                    image = extract(image_path)
                    images.append(image)
                    target.append(int(folder_name[1:]))
    return images , target                

A , B = Lecture_fichiers("archive (1)")
print(len(A),len(B))

400 400


In [18]:
def renameFile(racine):
    for folder in os.listdir(racine):
        pathFolder = os.path.join(racine, folder)
        for img in os.listdir(pathFolder):
            os.rename(pathFolder+'/'+img, pathFolder+'/'+img[0:len(img)-4]+'_{}'.format(folder) + '.pgm')

In [19]:
renameFile('archive (1)')

3- DÉFINITION DE LA FONCTION QUI PREND UNE MATRICE DE PIXELS ET RENVOIT LA MATRICES LBP BASIQUE ASSOCIÉE

UTILISATION DE CETTE FONCTION POUR REPRÉSENTER LES MATRICES LBP ISSUES DES MATRICES DE PIXELS PRÉCÉDENTES

In [5]:
def LBP_function (image):
    longueur , largeur = image.shape[0] , image.shape[1]
    lbp_image = np.zeros((longueur - 2 , largeur - 2),int)
    for i in range(1 , longueur -1):
        for j in range(1 , largeur -1):
            pixels_voisins = [image[i-1][j-1] , image[i-1][j] , image[i-1][j+1] , image[i][j+1],
                                  image[i+1][j+1] , image[i+1][j] , image[i+1][j-1] , image[i][j-1]]
            for n in range(len(pixels_voisins)):
                if pixels_voisins[n] >= image[i][j]:
                    lbp_image[i-1][j-1] = lbp_image[i-1][j-1]+2**n 
    return lbp_image            

In [6]:
def createPGM(matrix):
    with open('image.pgm', 'wb') as f:
        f.write(b'P5\n')
        f.write('{} {}\n'.format(len(matrix),len(matrix[0])).encode())
        f.write(b'255\n')
        f.write(matrix.astype(np.uint8).tobytes())

In [7]:
createPGM(LBP_function( extract('archive (1)/s1/1.pgm')))

In [8]:

def LBP_function_final(liste_imag):
    list_lbp_img = []
    for imag in liste_imag:
        lbp = LBP_function(imag)
        list_lbp_img.append(lbp)
    return list_lbp_img

RS = LBP_function_final(A)


4- DEFINITION DE LA FONCTION QUI Prend UNE MATRICE IMAGE LBP ET RENVOIE SON HISTOGRAMME

(on utilisera le fénètrage en précisant la dimension en paramètre aussi)

APPLICATIONDE LA FONCTION HISTOGRAMME SUR TOUTES NOS IMAGES LBP AFIN D'OBTENIR L'HISTOGRAMME DE CHACUNE

In [9]:
def histogramme(img_lbp , window = 10):
    hist_courant = []
    longueur , largeur = img_lbp.shape[0] , img_lbp.shape[1]
    for i in range(0 , longueur-window+1 , window):
        for j in range(0 , largeur-window+1 , window):
            hist = [0]*256
            for x in range(i , i+window):
                for y in range(j , j+window):
                    val = img_lbp[x][y]
                    hist[val] += 1
            hist /=np.sum(hist)
            hist_courant.append(hist)
    total_hist = list(hist_courant[0])
    for h in range(1 , len(hist_courant)):
        total_hist += list( hist_courant[h])
    return total_hist


In [10]:


def histigramme_final(liste_img_lbp ,  window_ = 10):
    liste_hist = []
    for img_lbp in liste_img_lbp:
        liste_hist.append(histogramme(img_lbp , window_ ))
    return liste_hist

In [11]:

RS_hist = histigramme_final(RS, 10)
print(len(RS_hist[0]))

25344


5- DÉFINITION DU JEU DE DONNÉES

histogrammes comme caractéristiques et numéro de la personne 

dont provient l'histogramme comme étiquette puis séparation en train et test

In [12]:
X , Y = RS_hist , B
test_size = 0.2
X_train, X_test , Y_train , Y_test= train_test_split(X , Y ,test_size=test_size, random_state=0 )
print(len(X_train))

320


6- DÉFINITION De DEUX MODÈLES SVM , l'un utilisant le noyau RBF  et l'autre le noyau linéaire

In [13]:
model1 = SVC(kernel= 'rbf' , C=1)
model1.fit(X_train, Y_train)

model2 = SVC(kernel= 'linear' , C=1)
model2.fit(X_train, Y_train)

SVC(C=1, kernel='linear')

7- ÉVALUATION DES MODÈLES SUR LE JEU DE TEST

In [14]:
accuracy = model1.score(X_test, Y_test)
print('prédiction du modèle avec noyau RBF:\t',accuracy*100)

accuracy = model2.score(X_test, Y_test)
print('\nprédiction du modèle avec noyau Linéaire:',accuracy*100)

prédiction du modèle avec noyau RBF:	 33.75

prédiction du modèle avec noyau Linéaire: 96.25


In [26]:
print(model1.predict(X_test[0:10]))
print(model2.predict(X_test[:10]))
print(Y_test[:10])

[40 14 39 32 36 19 40 39 39  3]
[ 5 28 22 32 36 19  6 11 16 12]
[5, 28, 22, 32, 36, 19, 6, 11, 24, 12]


on constate pour un premier essai que le noyau linéaire a tendance à etre plus performant

cependant nous allons utiliser gridseacrhCV avec ces deux noyaux et avec des valeurs de C différentes

In [17]:
from sklearn.model_selection import GridSearchCV
svc = SVC()
parameters = {'kernel' : ['linear' , 'rbf'] , 'C' : [1 , 4 , 10 , 15]}
model_affine =GridSearchCV(svc , parameters ,cv = 3 , n_jobs = -1) 
model_affine.fit(X_train ,Y_train)

In [ ]:
print(model_affine.best_params_)
print(model_affine.score(X_test , Y_test)*100)

{'C': 1, 'kernel': 'linear'}
97.5


DEFINITION DU MODÈLE FINALE AVEC C = 1 ET NOYAU LINÉAIRE

In [15]:
model_final = SVC(kernel= 'linear' , C=1)
model_final.fit(X_train, Y_train)

    
    

SVC(C=1, kernel='linear')

In [16]:
print(model_final.predict(X_test[:15]))
print(Y_test[:15])

[ 5 28 22 32 36 19  6 11 16 12 26  5  8 22  4]
[5, 28, 22, 32, 36, 19, 6, 11, 24, 12, 26, 5, 8, 22, 4]


In [17]:
accuracy = model2.score(X_test, Y_test)
print('\nprédiction du modèle avec noyau Linéaire:',accuracy*100)


prédiction du modèle avec noyau Linéaire: 96.25


In [18]:
file_name = 'final_model.sav'
pickle.dump(model2 , open(file_name , 'wb'))


In [19]:
model = pickle.load(open(file_name , 'rb'))


PREDICTION POUR UNE NOUVELLE DONNÉE

In [24]:
def prediction(image, model, window = 10):
    data = extract(image)
    lbp = LBP_function(data)
    histo = histogramme(lbp , window = 10)
    histo = [histo]
    rst = model.predict(histo)
    return rst

In [25]:
m = prediction('archive (1)/s5/2.pgm', model_final)
print(m)

[5]
